http://mxnet.io/tutorials/gluon/1-ndarray.html

NumpyのArrayと似た多次元配列が提供されている
- CPU, GPUでの非同期処理をサポート
- 自動微分をサポート

# Getting started

最初にインポート

In [1]:
import mxnet as mx
import mxnet.ndarray as nd

値なしで初期化

In [2]:
x = nd.empty(shape=(6,4))
print(x)

# print(x.asnumpy())  # 2017/7/16時点のリリースバージョンだとprint文で要素は見えないため、こちらでチェックする必要あり。


[[  5.62842689e-22   4.58546896e-41   1.02385242e-36   0.00000000e+00]
 [  1.63951920e-43   1.38728548e-43   1.45735040e-43   4.48415509e-44]
 [  1.42932443e-43   1.47136339e-43   1.51340234e-43   1.41531145e-43]
 [  4.48415509e-44   1.55544130e-43   1.59748025e-43   4.48415509e-44]
 [  1.40129846e-43   1.47136339e-43   1.59748025e-43   1.41531145e-43]
 [  1.38728548e-43   1.62550622e-43   1.55544130e-43   1.59748025e-43]]
<NDArray 6x4 @cpu(0)>


NNではよくやるように、乱数で初期化
（この場合だと標準正規分布で初期化）

In [3]:
x = nd.random_normal(shape=(6,4))
print(x)


[[ 2.21220636  1.16307867  0.7740038   0.48380461]
 [ 1.04344046  0.29956347  1.18392551  0.15302546]
 [ 1.89171135 -1.16881478 -1.23474145  1.55807114]
 [-1.771029   -0.54594457 -0.45138445 -2.35562968]
 [ 0.57938355  0.54144019 -1.85608196  2.67850661]
 [-1.9768796   1.25463438 -0.20801921 -0.54877394]]
<NDArray 6x4 @cpu(0)>


Numpyと同様に`.shape`で次元の取得が可能

In [4]:
print(x.shape)

(6, 4)


サイズ（要素数）も取得可能、格納された値の数により、アレイがどのくらいメモリを占めているかがわかる

In [5]:
print(x.size)

24


# Operations
多くの演算をサポート

In [6]:
y = nd.random_normal(shape=(6,4))
c = x + y
print(c)


[[ 2.45662808  0.4820143   0.73684311  0.34848878]
 [ 0.55569053  0.67679477  1.16130829  0.56318992]
 [ 2.46632552 -0.59754658  0.23138475 -1.19989157]
 [-1.08473861  0.53033543 -0.09642342 -2.96976233]
 [ 1.65255308  2.37220502 -1.73590732  1.53170013]
 [-2.94798994  1.30847239 -0.98371583 -3.05625463]]
<NDArray 6x4 @cpu(0)>


# In-place operations
上記だとx+yの値としてcを宣言したが、メモリ利用の効率化のため、すでに割り当てられているメモリを再利用できるin plane での演算を利用することが多い  
MXNetではスライスを用いてこれを表現

In [7]:
result = nd.zeros(shape=(6,4))
result[:] = x+y  # スライスで宣言
print(result)


[[ 2.45662808  0.4820143   0.73684311  0.34848878]
 [ 0.55569053  0.67679477  1.16130829  0.56318992]
 [ 2.46632552 -0.59754658  0.23138475 -1.19989157]
 [-1.08473861  0.53033543 -0.09642342 -2.96976233]
 [ 1.65255308  2.37220502 -1.73590732  1.53170013]
 [-2.94798994  1.30847239 -0.98371583 -3.05625463]]
<NDArray 6x4 @cpu(0)>


xを再利用するつもりがなければ、x自体に割り当ててしまえば良い

In [8]:
x[:] = x + y
print(result)


[[ 2.45662808  0.4820143   0.73684311  0.34848878]
 [ 0.55569053  0.67679477  1.16130829  0.56318992]
 [ 2.46632552 -0.59754658  0.23138475 -1.19989157]
 [-1.08473861  0.53033543 -0.09642342 -2.96976233]
 [ 1.65255308  2.37220502 -1.73590732  1.53170013]
 [-2.94798994  1.30847239 -0.98371583 -3.05625463]]
<NDArray 6x4 @cpu(0)>


ただしこれはx = x + y とは異なる点に注意！　スライスを用いなければxに対し新しいメモリが割り当てられ、そこへの参照が紐づけられる

# Slicing
馬鹿げてるほどたくさんのスライシングの方法がある  

In [9]:
x[3,2] = 9.0
print(x[3])


[-1.08473861  0.53033543  9.         -2.96976233]
<NDArray 4 @cpu(0)>


特定の箇所の値の変更

In [10]:
x[2:4]  # xの3,4行目をスライス(当然だが1行目は添え字0)
print(x[2:4])


[[ 2.46632552 -0.59754658  0.23138475 -1.19989157]
 [-1.08473861  0.53033543  9.         -2.96976233]]
<NDArray 2x4 @cpu(0)>


# Weild multi-dimensional slicing
どの軸方向についても、任意の範囲でスライシングできる

In [11]:
x[2:4,1:3] = 5.0  # 3-4行、2-3列の値を5に。
print(x)


[[ 2.45662808  0.4820143   0.73684311  0.34848878]
 [ 0.55569053  0.67679477  1.16130829  0.56318992]
 [ 2.46632552  5.          5.         -1.19989157]
 [-1.08473861  5.          5.         -2.96976233]
 [ 1.65255308  2.37220502 -1.73590732  1.53170013]
 [-2.94798994  1.30847239 -0.98371583 -3.05625463]]
<NDArray 6x4 @cpu(0)>


# Converting from MXNet NDArray to NumPy
簡単に変更可能  
PyTorchと違って、メモリを共有しないことに注意

In [12]:
a = nd.ones(shape=(5))
print(a)


[ 1.  1.  1.  1.  1.]
<NDArray 5 @cpu(0)>


In [13]:
b = a.asnumpy()
print(b)


[ 1.  1.  1.  1.  1.]


In [14]:
b[0] = 2
print(b)
print(a)  # メモリが共有されないことの確認

[ 2.  1.  1.  1.  1.]

[ 1.  1.  1.  1.  1.]
<NDArray 5 @cpu(0)>


# Converting from NumPy Array to MXNet NDArray
こちらも素直。

In [15]:
c = nd.array(b)
print(c)


[ 2.  1.  1.  1.  1.]
<NDArray 5 @cpu(0)>


# Managing context
MXNetでは全てのアレイについて、CPUやGPU（たくさんのコアのうちの一つ）と、コンテクストがある。  
インテリジェントにアレイをコンテクストへと紐づけることでデバイス間のデータ転送を最小化できる  

In [16]:
d = nd.array(b, mx.cpu())

In [17]:
print(d) # だいたいcpu0に紐づけられてるはず


[ 2.  1.  1.  1.  1.]
<NDArray 5 @cpu(0)>


他のコンテクストに`copyto()`メソッドでコピー可能

In [18]:
e = d.copyto(mx.cpu(1)) # cpu1へコピー

In [19]:
print(e)


[ 2.  1.  1.  1.  1.]
<NDArray 5 @cpu(1)>


# Watch out !
もし`d`がすでにセカンドGPU(`mx.gpu(1)`)に存在していて、`d.copyto(mx.gpu(1))`をコールするとどうなるか？  
この場合、変数がすでに存在していたとしてもコピーを生成してしまう  

間違ったコンテクストに存在する変数をコピーしたいときは`as_in_context()`を利用する。  
すでに存在していた場合はなにもしない。

In [20]:
f = d.as_in_context(mx.cpu(0)) #と言ってもチェック方法ない気が
print(f)


[ 2.  1.  1.  1.  1.]
<NDArray 5 @cpu(0)>


In [21]:
# GPUない環境でやるとどうなる？
g = d.copyto(mx.gpu()) 
print(g)

MXNetError: [09:06:24] src/ndarray/ndarray.cc:383: GPU is not enabled

Stack trace returned 10 entries:
[bt] (0) /mxnet/python/mxnet/../../lib/libmxnet.so(_ZN4dmlc15LogMessageFatalD1Ev+0x3c) [0x7fd2f5f76d2c]
[bt] (1) /mxnet/python/mxnet/../../lib/libmxnet.so(_ZN5mxnet10CopyFromToERKNS_7NDArrayEPS0_i+0x30b) [0x7fd2f6c3786b]
[bt] (2) /mxnet/python/mxnet/../../lib/libmxnet.so(+0xebb906) [0x7fd2f6bb9906]
[bt] (3) /mxnet/python/mxnet/../../lib/libmxnet.so(_Z20ImperativeInvokeImplRKN5mxnet7ContextERKN4nnvm9NodeAttrsEPSt6vectorINS_7NDArrayESaIS8_EESB_+0x100) [0x7fd2f6f92570]
[bt] (4) /mxnet/python/mxnet/../../lib/libmxnet.so(MXImperativeInvoke+0x217) [0x7fd2f6f93037]
[bt] (5) /usr/lib/python3.5/lib-dynload/_ctypes.cpython-35m-x86_64-linux-gnu.so(ffi_call_unix64+0x4c) [0x7fd318483e20]
[bt] (6) /usr/lib/python3.5/lib-dynload/_ctypes.cpython-35m-x86_64-linux-gnu.so(ffi_call+0x2eb) [0x7fd31848388b]
[bt] (7) /usr/lib/python3.5/lib-dynload/_ctypes.cpython-35m-x86_64-linux-gnu.so(_ctypes_callproc+0x